In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'unique_beer_drinkers'
AUTHOR = 'will'

In [3]:
task = Task(
    id=TASK_ID,
    category="First-order logic",
    name="People who like a unique set of beer",
    description="Find all people whose preferred set of beers is distinct from each other person's preferred set",
    plan=[
        Plan(id="collect", description="preferred set of beers"),
        Plan(id="iter", description="all people"),
        Plan(id="compare", description="distinct from each other person's preferred set")
    ],
    sample_input={
        "likes": [
            {"name": "will", "beer": "ipa"},
            {"name": "will", "beer": "lager"},
            {"name": "scott", "beer": "ipa"},
            {"name": "scott", "beer": "stout"},
            {"name": "gleb", "beer": "ipa"},
            {"name": "gleb", "beer": "stout"},
            {"name": "fred", "beer": "ipa"},
            {"name": "fred", "beer": "lager"},
            {"name": "fred", "beer": "stout"},
        ]
    },
    sample_output=["will", "fred"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [13]:
q = replace(prototype,
    language='q',
    source='''likes_per_person: `name xgroup likes;
counts: count each group likes_per_person;
unique_beer_drinkers: 
  (select name from likes_per_person where beer in\: where[counts=1])
  `name''').load_plan()
q.execute(task)
q.save()

In [11]:
q.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "q", "plan": {}, "source": "likes_per_person:…

In [95]:
r = replace(prototype,
    language='r',
    source='''unique_beer_drinkers <- function(likes) {
  likes %>%
    group_by(name) %>%
    summarize(beer_set = list(sort(unique(beer)))) %>%
    add_count(beer_set, name = 'num_people_likes') %>%
    filter(num_people_likes == 1) %>%
    pull(name)
}''').load_plan()
r.execute(task)
r.save()

`summarise()` ungrouping output (override with `.groups` argument)


In [96]:
r.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "r", "plan": {}, "source": "unique_beer_drink…

In [17]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "sql", "plan": {}, "source": "SELECT DISTINCT…

In [10]:
datalog1 = replace(
    prototype,
    language='datalog',
    implementation='count',
    source='''unique_beer_drinkers(Name) :-
  likes(_, Name),
  0 = count : {
    likes(_, Other),
    Other != Name,
    Shared_count = count : { likes(Beer, Other), likes(Beer, Name) },
    count : likes(_, Other) = Shared_count,
    count : likes(_, Name) = Shared_count
  }.''')
datalog1.execute(task)
datalog1.save()

In [25]:
datalog2 = replace(
    prototype,
    language='datalog',
    implementation='neg_exist',
    source='''.decl differ(a:symbol, b:symbol)
differ(A, B) :- likes(Beer, A), likes(_, B), !likes(Beer, B).
differ(A, B) :- likes(_, A), likes(Beer, B), !likes(Beer, A).

.decl exists_same(a:symbol)
exists_same(Name) :- 
  likes(_, Other), 
  likes(_, Name), 
  Name != Other, 
  !differ(Name, Other).

unique_beer_drinkers(Name) :- 
  likes(_, Name), 
  !exists_same(Name).''')
datalog2.execute(task)
datalog2.save()

In [26]:
datalog2.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "datalog", "plan": {}, "source": ".decl diffe…

In [14]:
pandas = replace(
    prototype,
    language='python-pandas',
    author='will',
    source='''def unique_beer_drinkers(likes):
  likes_per_person = (likes
    .groupby('name')
    .beer.unique().map(set)
    .reset_index())

  def check_not_exists(row):
    other_people = likes_per_person[
      likes_per_person.name != row['name']]
    return not (other_people.beer == row['beer']).any()
  
  unique_drinkers = likes_per_person[
    likes_per_person.apply(check_not_exists, axis=1)]
  return unique_drinkers.name.tolist()''')
pandas.execute(task)
pandas.save()

In [15]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-pandas", "plan": {}, "source": "def u…

In [10]:
python_functional = replace(
    prototype,
    language='python-functional',
    source='''def unique_beer_drinkers(likes):
  people = set([row['name'] for row in likes])
  likes_per_person = {
    name: set([row['beer'] for row in likes if row['name'] == name])
    for name in people
  }
    
  return [
    name
    for name, beers in likes_per_person.items()
    if not any([
      other_name != name and beers == other_beers
      for other_name, other_beers in likes_per_person.items()
    ])
  ]''')
python_functional.execute(task)
python_functional.save()

In [11]:
python_functional.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-functional", "plan": {}, "source": "d…

In [12]:
python_imperative = replace(
    prototype,
    language='python-imperative',
    source='''def unique_beer_drinkers(likes):
  likes_per_person = defaultdict(set)
  for row in likes:
    likes_per_person[row['name']].add(row['beer'])
    
  unique = []
  for p1, p1_likes in likes_per_person.items():
    is_unique = True
    for p2, p2_likes in likes_per_person.items():
      if p1 == p2:
        continue
        
      if p1_likes == p2_likes:
        is_unique = False
        break
    if is_unique:
      unique.append(p1)
      
  return unique''')
python_imperative.execute(task)
python_imperative.save()

In [13]:
python_imperative.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-imperative", "plan": {}, "source": "d…